In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


In [2]:
from google.colab import drive
drive.mount('/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /drive


In [3]:
import os
os.chdir('/drive/My Drive')

In [4]:
#pip install tensorflow==1.14.0

In [4]:
import warnings

In [5]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob


import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [6]:
IMAGE_SIZE = [224, 224]

In [7]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58892288/58889256 [==============================] - 2s 0us/step


In [8]:
# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

In [9]:
x = Flatten()(vgg.output)
prediction = Dense(4, activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [10]:
from keras import optimizers


sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9)
model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [11]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [12]:
# Data Augmentation
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [13]:
# Make sure you provide the same target size as initialied for the image size
train_set = train_datagen.flow_from_directory('/drive/My Drive/IMAGE_RECOGNITION/GADGETS/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 264 images belonging to 4 classes.


In [14]:
test_set = test_datagen.flow_from_directory('/drive/My Drive/IMAGE_RECOGNITION/GADGETS/train',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 264 images belonging to 4 classes.


In [17]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau

#lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

#num_epochs = 1000
#num_batch_size = 32

checkpoint = ModelCheckpoint(filepath='/drive/My Drive/IMAGE_RECOGNITION/GADGETS/my_model.h5', 
                               verbose=1, save_best_only=True)

callbacks = [checkpoint, lr_reducer]

start = datetime.now()

model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=15,
  steps_per_epoch=500,
  validation_steps=15,
    callbacks=callbacks ,verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/15
  1/500 [..............................] - ETA: 8:02 - loss: 3.6045 - accuracy: 0.7656

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


500/500 [==============================] - 321s 641ms/step - loss: 3.4617 - accuracy: 0.7739 - val_loss: 1.6482 - val_accuracy: 0.8509

Epoch 00001: val_loss improved from inf to 1.64822, saving model to /drive/My Drive/IMAGE_RECOGNITION/GADGETS/my_model.h5
Epoch 2/15
500/500 [==============================] - 320s 640ms/step - loss: 0.8539 - accuracy: 0.9441 - val_loss: 0.0000e+00 - val_accuracy: 0.9757

Epoch 00002: val_loss improved from 1.64822 to -0.00000, saving model to /drive/My Drive/IMAGE_RECOGNITION/GADGETS/my_model.h5
Epoch 3/15
500/500 [==============================] - 314s 629ms/step - loss: 0.5390 - accuracy: 0.9651 - val_loss: 1.9171 - val_accuracy: 0.9560

Epoch 00003: val_loss did not improve from -0.00000
Epoch 4/15
500/500 [==============================] - 320s 639ms/step - loss: 0.5533 - accuracy: 0.9635 - val_loss: 0.9586 - val_accuracy: 0.9682

Epoch 00004: val_loss did not improve from -0.00000
Epoch 5/15
500/500 [==============================] - 320s 641ms/s

In [19]:

from tensorflow import keras
model = keras.models.load_model('/drive/My Drive/IMAGE_RECOGNITION/GADGETS/my_model.h5',)

In [20]:
# Part 3 - Making new predictions
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/drive/My Drive/IMAGE_RECOGNITION/GADGETS/test/watch/1.jpg', target_size = (224,224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

In [21]:
train_set.class_indices

{'backpack': 0, 'footwear': 1, 'glasses': 2, 'watch': 3}

In [22]:
result


array([[0., 0., 0., 1.]], dtype=float32)

In [23]:
train_set.class_indices
if result[0][0] == 1:
  prediction = 'backpack'
  print(prediction)

elif result[0][1] == 2:
  prediction = 'footwear'
  print(prediction)

elif result[0][2] == 3:
  prediction = 'glasses'
  print(prediction)

else:
  prediction = 'watch'
  print(prediction)

watch
